In [ ]:
# Importa as bibliotecas

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn

: 

In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

engine = sqlalchemy.create_engine(cnx)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'consulta' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from consulta;" 
df = pd.read_sql_query(query, cnx)

df

In [ ]:
#campos de texto

#declare esta variável para usar na consulta de campos em branco
flag=''

# Cria widgets interativos para o usuário inserir ou selecionar dados:

id_fila = pn.widgets.IntInput(
    name="ID Fila",
    value=None,
    placeholder='Digite o ID da Fila',
    disabled=False
)

id_triagem = pn.widgets.IntInput(
    name="ID Triagem",
    value=None,
    placeholder='Digite o ID da Triagem',
    disabled=False
)

cpf_consulta = pn.widgets.TextInput(
    name = "Paciente",
    value='',
    placeholder='Digite o CPF',
    disabled=False
)

cpf_tec = pn.widgets.TextInput(
    name="Técnico",
    value='',
    placeholder='Digite o CPF',
    disabled=False
)

cpf_enf = pn.widgets.TextInput(
    name="Enfermeiro",
    value='',
    placeholder='Digite o CPF',
    disabled=False
)

cpf_med = pn.widgets.TextInput(
    name = "Médico",
    value='',
    placeholder='Digite o CPF',
    disabled=False
)

dataconsulta = pn.widgets.DatePicker(
    name='Data da consulta',
    disabled=False
)


In [ ]:
# Cria quatro botões para as ações principais da aplicação CRUD:
# Consultar, Inserir, Excluir e Atualizar registros no banco de dados


buttonConsultar = pn.widgets.Button(name='Consultar', button_type='default')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='default')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='default')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='default')

In [ ]:
def queryAll():
    """
    Consulta todos os registros da tabela 'consulta' no banco de dados e retorna
    um widget Tabulator para exibição interativa dos dados.

    Returns:
        pn.widgets.Tabulator: Widget que exibe a tabela com todos os dados da tabela 'consulta'.
    """
    query = f"select * from consulta"
    df = pd.read_sql_query(query, cnx)
    return pn.widgets.Tabulator(df)


def on_consultar():
    """
    Consulta registros na tabela consulta
    filtrando pelo CPF do paciente informado. Se o CPF estiver vazio, retorna todos.
    """
    try:
        cpf_val = cpf_consulta.value_input or ""

        if cpf_val == "":
            query = "SELECT * FROM consulta"
            df = pd.read_sql_query(query, cnx)
        else:
            query = "SELECT * FROM consulta WHERE cpf_paciente = %s"
            df = pd.read_sql_query(query, cnx, params=(cpf_val,))

        return pn.widgets.Tabulator(df)

    except Exception as e:
        return pn.pane.Alert(f"Erro ao consultar: {e}")


def on_inserir():
    """
    Insere um novo registro na tabela 'consulta' com os dados fornecidos.
    """
    try:
        cpf_paciente_val = cpf_consulta.value_input.strip()
        cpf_tec_val      = cpf_tec.value_input.strip() if cpf_tec.value_input.strip() else None
        cpf_enf_val      = cpf_enf.value_input.strip() if cpf_enf.value_input.strip() else None
        cpf_med_val      = cpf_med.value_input.strip() if cpf_med.value_input.strip() else None
        data_consulta_val = dataconsulta.value
        id_fila_val       = id_fila.value
        id_triagem_val    = id_triagem.value
        status_val       = "Agendada"

        if not cpf_paciente_val:
            return pn.pane.Alert("CPF do paciente é obrigatório!")

        cursor = con.cursor()
        
        cursor.execute("""
            INSERT INTO consulta (
                id_fila, cpf_enfermeiro, cpf_tecnico, cpf_paciente, 
                id_triagem, cpf_medico, data_consulta, status
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            id_fila_val, cpf_enf_val, cpf_tec_val, cpf_paciente_val,
            id_triagem_val, cpf_med_val, data_consulta_val, status_val
        ))
        
        con.commit()
        cursor.close()
        return queryAll()

    except Exception as e:
        return pn.pane.Alert(f"Não foi possível inserir: {str(e)}")


def on_atualizar():
    """
    Atualiza um registro da tabela consulta identificado pelo CPF do paciente.
    """
    try:
        cpf_paciente_val = cpf_consulta.value_input.strip()
        cpf_tec_val      = cpf_tec.value_input.strip() if cpf_tec.value_input.strip() else None
        cpf_enf_val      = cpf_enf.value_input.strip() if cpf_enf.value_input.strip() else None
        cpf_med_val      = cpf_med.value_input.strip() if cpf_med.value_input.strip() else None
        data_consulta_val = dataconsulta.value
        id_fila_val       = id_fila.value
        id_triagem_val    = id_triagem.value
        status_val       = "Agendada"

        if not cpf_paciente_val:
            return pn.pane.Alert("CPF do paciente é obrigatório para atualizar!")

        cursor = con.cursor()
        cursor.execute("""
            UPDATE consulta SET 
                id_fila = %s, cpf_enfermeiro = %s, cpf_tecnico = %s, 
                id_triagem = %s, cpf_medico = %s, data_consulta = %s, status = %s
            WHERE cpf_paciente = %s
        """, (
            id_fila_val, cpf_enf_val, cpf_tec_val, id_triagem_val,
            cpf_med_val, data_consulta_val, status_val, cpf_paciente_val
        ))
        
        con.commit()
        cursor.close()
        return queryAll()

    except Exception as e:
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}')


def on_excluir():
    """
    Exclui o registro da tabela 'consulta' com o CPF do paciente informado.
    """
    try:
        cpf_paciente_val = cpf_consulta.value_input.strip()

        if not cpf_paciente_val:
            return pn.pane.Alert("CPF do paciente é obrigatório para excluir!")

        cursor = con.cursor()
        cursor.execute("DELETE FROM consulta WHERE cpf_paciente = %s", (cpf_paciente_val,))
        
        con.commit()
        cursor.close()
        return queryAll()

    except Exception as e:
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}')

In [ ]:
visivel = pn.Column(visible=True)  # antes de table_creator()

def table_creator(cons, ins, atu, exc):
    result = None
    if cons:
        result = on_consultar()
    if ins:
        result = on_inserir()
    if atu:
        result = on_atualizar()
    if exc:
        result = on_excluir()

    if result is not None:
        visivel[:] = [result]
        visivel.visible = True
    return result


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

interactive_table = pn.bind(table_creator, buttonConsultar, buttonInserir, buttonAtualizar, buttonExcluir)

In [ ]:
# Monta o layout da interface com Panel:
# - Coluna esquerda com o título, os campos de entrada e os botões de ação
# - Coluna direita com a tabela interativa que mostra os dados do banco
# O método `.servable()` permite que essa interface seja exibida ao rodar o Panel server

visivel = pn.Column(
            interactive_table,
            styles={'background': 'white',
                    'border-radius': '10px',
                    'padding': '10px',
                    'box-shadow': '0px 0px 10px black',
                    'margin-top' : '0px',
            },
            visible = False
        )


pn.extension('tabulator', 
    raw_css=[
        '''
        body {
            background-color: #2072D0;
        }
        
        .fundo_branco{
            background-color: white;
        }
        
        .tabulator{
            border-radius: 5px;
            box-shadow: 0px 0px 5px black;
        }
        '''
    ]
)

pn.Row(
    pn.pane.Markdown(
        '**Unidade Básica de Saúde**', 
        
        styles={'font-family' : 'Trebuchet MS',
                'font-size': '64px',
                'margin' : '-75px',
                'margin-left' : '100px'}
    ),
    
    pn.layout.HSpacer(),
    pn.pane.Markdown(
      '**Tela Consulta**',
      styles={'font-family' : 'Trebuchet MS',
                'font-size': '64px',
                'margin' : '-75px',
                'text-align' : 'center'}
    ),
    pn.layout.HSpacer(),
        
    styles = {'background-color': 'white',
              'width' : '100%',
              'height' : '75px',
              'box-shadow' : '0px 0px 15px black'}
).servable()

pn.GridBox( 
    pn.Column(
        pn.Row(
            id_fila,
            id_triagem,
        ),
        pn.Row(
            cpf_consulta,
            cpf_enf,
        ),
        pn.Row(
            cpf_med,
            dataconsulta,
        ),
        pn.Row(
            cpf_tec
        ),
        styles = {'margin-left' : 'auto',
                  'margin-right' : 'auto',
                  'margin-top' : '0px'}
    ),
    

    pn.Row(
        pn.Row(buttonConsultar),
        pn.Row(buttonInserir),
        pn.Row(buttonAtualizar),
        pn.Row(buttonExcluir),

        
        styles = {'margin': 'auto',
                  'margin-top': '0px'},
    ),
    
    styles = {'background-color': 'white', 
              'margin': 'auto', 
              'width': '715px', 
              'height': '550px', 
              'margin-top': '50px',
              'border-radius': '15px',
              'box-shadow' : '0px 0px 13px black'}
).servable()

pn.Row(
    visivel,
    
    styles = {'margin': 'auto',
            'margin-top': '100px'},
).servable()